In [23]:
from support_functions.load_data import load_datasets_with_annotations as loading
from support_functions.data_transformations import convert_annotation_dataset

import json
import time 
import copy


In [24]:


def createEntry(time, stories, redundancyType, triggers, targets, contains): 
 
    json_entry = {
    "elipsedTimeNs": time,
    "relatedStories": stories,
    "mainPartRedundancies": {
    "partialRedundancy": redundancyType['Partial']['Main Part'],
    "fullRedundancy": redundancyType['Full']['Main Part'],
    "mainPartExplanationOfRedundancy": "",
    "pairsOfTriggersRedundancies": triggers['Main Part'],
    "pairsOfTargetsRedundancies": targets['Main Part'],
    "pairsOfContainsRedundancies": contains['Main Part']
    },
    "benefitRedundancies": {
    "partialRedundancy": redundancyType['Partial']['Benefit'],
    "fullRedundancy": redundancyType['Full']['Benefit'],
    "benefitExplanationOfRedundancy": "",
    "pairsOfTriggersRedundancies": triggers['Benefit'],
    "pairsOfTargetsRedundancies": targets['Benefit'],
    "pairsOfContainsRedundancies": contains['Benefit']
    }
    }
    return json_entry





In [25]:
not_processed_datasets: dict[str, list] = loading()
datasets, ignored_items = convert_annotation_dataset(not_processed_datasets)

datasets.keys()

nr_dataset = 'g22'

user_stories = datasets[nr_dataset]
nr_stories = len(user_stories)

references = ['Targets', 'Triggers', 'Contains']
parts = ['Main Part', 'Benefit']


mainpart = 0
benefit = 0
total = 0
output = []

for i in range(nr_stories):
    story1 = user_stories[i]
    
    for j in range(i+1, nr_stories):
        start = time.time_ns()
        redundancy_type = {
                'Partial': {'Main Part': False, 
                            'Benefit': False},
                'Full': {'Main Part': True, 
                            'Benefit': True}, 
        }    
        redundancies = {
                'Triggers': {'Main Part': [], 
                             'Benefit': []
                             },
                'Targets': {'Main Part': [], 
                             'Benefit': []
                             },
                'Contains': {'Main Part': [], 
                             'Benefit': []
                             }
        }

        story2 = user_stories[j]

        for ref in references:
            for part in parts: 
                intersection = [val for val in story1[ref][part] if val in story2[ref][part]]
            
                if len(intersection) > 0 and ref == 'Targets':
                    redundancy_type['Partial'][part] = True
                    

                if redundancy_type['Partial'][part] == True:
                    redundancies[ref][part] = intersection
                    


                if len(intersection) != len(story1[ref][part]) or len(intersection) != len(story2[ref][part]):
                    redundancy_type['Full'][part] = False

        for part in parts: 
            redundancy_type['Full'][part] = redundancy_type['Full'][part] and redundancy_type['Partial'][part]

        entry = createEntry(time.time_ns() - start, [story1['USID'], story2['USID']],redundancy_type, redundancies['Triggers'], redundancies['Targets'], redundancies['Contains'])
        output.append(copy.deepcopy(entry))
       
        #if redundancy_type['Partial']['Main Part'] or redundancy_type['Partial']['Benefit']:
        #    print(entry)
        #    print(story1)
        #    print(story2)
        #    print("----")


        if  redundancy_type['Partial']['Main Part']:
            mainpart += 1        
        if  redundancy_type['Partial']['Benefit']:
            benefit +=1
        if  redundancy_type['Partial']['Main Part'] or  redundancy_type['Partial']['Benefit']:
            total += 1
        

path = "results_graph_" + nr_dataset +".json"

temp = copy.deepcopy(output)

object = json.dumps(output,indent = 4)
with open(path,"w") as outfile:
    outfile.write(object)

print("Main Part", mainpart)
print("Benefit", benefit)
print("Total", total)
   
            

Main Part 35
Benefit 32
Total 54


In [26]:
from support_functions.excel_helper import prepaire_excel_data

In [27]:
excel_data_graph_approach = prepaire_excel_data("Framework", "\\src_graph_approach")
display(excel_data_graph_approach.head(3))

,Identifier,Project Identifier,USID 1,User Story 1 Text with Markers,USID 2,User Story 2 Text with Markers,Total Redundancy Clause,Main Part Redundancy Clause,Benefit Part Redundancy Clause,Main Part Partial,Main Part Full,Benefit Partial,Benefit Full
1,g19user_story_1021_AND_user_story_1036,g19,1021,"As an OlderPerson, I want to #use# #ALFRED# as...",1036,"As an OlderPerson, I want to #use# #ALFRED# to...",2,2,0,True,False,False,False
2,g19user_story_1021_AND_user_story_1056,g19,1021,"As an OlderPerson, I want to #use# #ALFRED# as...",1056,"As an OlderPerson, I want to #use# #ALFRED# fo...",2,2,0,True,False,False,False
3,g19user_story_1021_AND_user_story_1057,g19,1021,"As an OlderPerson, I want to #use# #ALFRED# as...",1057,"As an OlderPerson, I want to #use# #ALFRED# fo...",2,2,0,True,False,False,False


In [28]:
systementical_difference_main_part: list[dict] = []
systementical_difference_benefit: list[dict] = []
# & (excel_data_graph_approach['Main Part Redundancy Clause**'].astype(int) == 1)
additional_pairs_main_part = copy.deepcopy(excel_data_graph_approach)
additional_pairs_benefit = copy.deepcopy(excel_data_graph_approach)

for item in temp:
    if item['mainPartRedundancies']['partialRedundancy'] or item['mainPartRedundancies']['fullRedundancy']:
        usid1, usid2 = item['relatedStories']
        filtered_rows = excel_data_graph_approach[
                        ((excel_data_graph_approach["USID 1"].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach["USID 2"].astype(int) == int(usid2))
                        & (excel_data_graph_approach['Main Part Redundancy Clause'].astype(int) >= 1))
                    ]
        
        filtered_rows2 = excel_data_graph_approach[
                        ((excel_data_graph_approach["USID 1"].astype(int) == int(usid2)) 
                        & (excel_data_graph_approach["USID 2"].astype(int) == int(usid1))
                        & (excel_data_graph_approach['Main Part Redundancy Clause'].astype(int) >= 1))
                    ]
        
        additional_pairs_main_part = additional_pairs_main_part[
                        ((additional_pairs_main_part["USID 1"].astype(int) != int(usid2)) 
                        & (additional_pairs_main_part["USID 2"].astype(int) != int(usid1))
                        & (additional_pairs_main_part['Main Part Redundancy Clause'].astype(int) >= 1))
                    ]
        
        additional_pairs_main_part = additional_pairs_main_part[
                        ((additional_pairs_main_part["USID 1"].astype(int) != int(usid1)) 
                        & (additional_pairs_main_part["USID 2"].astype(int) != int(usid2))
                        & (additional_pairs_main_part['Main Part Redundancy Clause'].astype(int) >= 1))
                    ]

        if filtered_rows.empty and filtered_rows2.empty:
            systementical_difference_main_part.append(item)
        
    if item['benefitRedundancies']['partialRedundancy'] or item['benefitRedundancies']['fullRedundancy']:
        usid1, usid2 = item['relatedStories']
        filtered_rows = excel_data_graph_approach[
                        (excel_data_graph_approach["USID 1"].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach["USID 2"].astype(int) == int(usid2))
                        & (excel_data_graph_approach['Benefit Part Redundancy Clause'].astype(int) >= 1)
                    ]
        filtered_rows2 = excel_data_graph_approach[
                        (excel_data_graph_approach["USID 1"].astype(int) == int(usid2)) 
                        & (excel_data_graph_approach["USID 2"].astype(int) == int(usid1))
                        & (excel_data_graph_approach['Benefit Part Redundancy Clause'].astype(int) >= 1)
                    ]
        
        additional_pairs_benefit = additional_pairs_benefit[
                        ((additional_pairs_benefit["USID 1"].astype(int) != int(usid2)) 
                        & (additional_pairs_benefit["USID 2"].astype(int) != int(usid1))
                        & (additional_pairs_benefit['Benefit Part Redundancy Clause'].astype(int) >= 1))
                    ]
        
        additional_pairs_benefit = additional_pairs_benefit[
                        ((additional_pairs_benefit["USID 1"].astype(int) != int(usid1)) 
                        & (additional_pairs_benefit["USID 2"].astype(int) != int(usid2))
                        & (additional_pairs_benefit['Benefit Part Redundancy Clause'].astype(int) >= 1))
                    ]
        if filtered_rows.empty and filtered_rows2.empty:
            systementical_difference_benefit.append(item)   

In [29]:
print(len(systementical_difference_main_part))

for item in systementical_difference_main_part:
    story1, story2 = item['relatedStories']
    for story in user_stories: 
        if story['USID'] == story1 or story['USID'] == story2 : 
            print(story['USID'], story['Text'])
            print(story['USID'], " : ", story['Targets'])
    print("---Next Pair ---")

0


In [30]:
additional_pairs_main_part = additional_pairs_main_part[ (additional_pairs_main_part['Main Part Redundancy Clause'].astype(int) >= 1)
                                    &(additional_pairs_main_part['Project Identifier'] == nr_dataset) ]

print( len(additional_pairs_main_part))

for index,row in additional_pairs_main_part.iterrows():
    for story in user_stories:
        if int(story['USID']) == int(row['USID 1']) or int(story['USID']) == int(row['USID 2']):
            print(story['USID'], story['Text'])
            print(story['USID'], story['Targets'])
    print("---Next Pair ---")


0


In [31]:
print(len(systementical_difference_benefit))

for item in systementical_difference_benefit:
    story1, story2 = item['relatedStories']
    for story in user_stories: 
        if story['USID'] == story1 or story['USID'] == story2 : 
            print(story['USID'], story['Text'])
            print(story['USID'], " : ", story['Targets'])
            
    print("---Next Pair ---")

0


In [ ]:
additional_pairs_benefit = additional_pairs_benefit[ (additional_pairs_benefit['Benefit Part Redundancy Clause'].astype(int) >= 1)
                                    &(additional_pairs_benefit['Project Identifier'] == nr_dataset) ]

print( len(additional_pairs_benefit))

for index,row in additional_pairs_benefit.iterrows():
    for story in user_stories:
        if int(story['USID']) == int(row['USID 1']) or int(story['USID']) == int(row['USID 2']):
            print(story['USID'], story['Text'])
            print(story['USID'], story['Targets'])
    print("---Next Pair ---")


KeyError: 'Benefit Part Redundancy Clause ***'